# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
# Install the plaidml backend
## DO THIS BEFORE IMPORTING KERAS OR TENSOR TO USE PLAIDML
import plaidml.keras

plaidml.keras.install_backend()

In [2]:
# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# Gets rid of the processor warning.
#os.environ['KMP_DUPLICATE_LIB_OK']='True' # haven't tried yet

In [3]:
# Libraries
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
import this

/opt/anaconda3/envs/ironhack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/ironhack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/ironhack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/ironhack/lib/python3.7/site-packages/tensorf

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [4]:
# display for notebooks
%matplotlib inline

In [7]:
# Versions
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("plaidml version:", plaidml.__version__)

numpy version: 1.18.1
pandas version: 1.0.1
plaidml version: 0.7.0


In [8]:
# creating a DF
tictactoe = pd.read_csv("tic-tac-toe.csv")

In [9]:
# checking the DF
tictactoe

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [12]:
# Converting categorical values to numeric
# creating dict of conversion
cat_to_num = {"x": 1, "o": 0, "b": -1}

In [13]:
# converting
tictactoe.replace(cat_to_num, inplace = True)

In [14]:
# checking
tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,0,0,1,0,0,True
1,1,1,1,1,0,0,0,1,0,True
2,1,1,1,1,0,0,0,0,1,True
3,1,1,1,1,0,0,0,-1,-1,True
4,1,1,1,1,0,0,-1,0,-1,True


In [15]:
X = np.array(tictactoe.drop(labels = "class", axis = 1))
y = np.array(tictactoe["class"])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [17]:
# your code here
# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
# Creating the Sequential model
model = Sequential([
  Dense(18, activation = "relu"),
  Dense(18, activation = "relu"),
  Dense(2, activation = "softmax")])

In [19]:
# Compiling the model
model.compile(
  optimizer = "adam",
  loss = "sparse_categorical_crossentropy",
  metrics = ["accuracy"])

INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


In [20]:
# Train the model.
history = model.fit(
  X_train,
  y_train,
  epochs = 50,
  batch_size = 40)

Epoch 1/50
766/766 [==============================] - 1s 1ms/step - loss: 0.6830 - acc: 0.5979
Epoch 2/50
766/766 [==============================] - 0s 91us/step - loss: 0.6498 - acc: 0.6240
Epoch 3/50
766/766 [==============================] - 0s 97us/step - loss: 0.6359 - acc: 0.6436
Epoch 4/50
766/766 [==============================] - 0s 99us/step - loss: 0.6230 - acc: 0.6540
Epoch 5/50
766/766 [==============================] - 0s 107us/step - loss: 0.6143 - acc: 0.6567
Epoch 6/50
766/766 [==============================] - 0s 94us/step - loss: 0.6075 - acc: 0.6606
Epoch 7/50
766/766 [==============================] - 0s 87us/step - loss: 0.6002 - acc: 0.6606
Epoch 8/50
766/766 [==============================] - 0s 86us/step - loss: 0.5935 - acc: 0.6684
Epoch 9/50
766/766 [==============================] - 0s 86us/step - loss: 0.5869 - acc: 0.6775
Epoch 10/50
766/766 [==============================] - 0s 86us/step - loss: 0.5793 - acc: 0.6762
Epoch 11/50
766/766 [==================

In [21]:
# Evaluate the model.
model.evaluate(
  X_test,
  y_test)

192/192 [==============================] - 1s 6ms/step


[0.39106981456279755, 0.84375]

In [22]:
# saving the model
model.save_weights("tic-tac-toe.model")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here
# Loading the model
model.load_weights("tic-tac-toe.model")


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here